In [1]:
import requests
from bs4 import BeautifulSoup as BS
import numpy as np
import pandas as pd
import re
import statsmodels.api as sm
import statsmodels.graphics as g
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import time
from jupyterthemes import jtplot
jtplot.style()

# Hide Code in Slideshow

In [2]:
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)

# MVP

## Read In

In [3]:
# Read in campaign-funds data from Problem 3
US = pd.read_csv('../data/us_rep_elections.csv')
US

,district,name,party,incumbent,winner,raised,spent
0,AL01,Jerry Carl,R,False,True,1971321,1859349
1,AL01,James Averhart,D,False,False,80095,78973
2,AL02,Barry Moore,R,False,True,650807,669368
3,AL02,Phyllis Harvey-Hall,D,False,False,56050,55988
4,AL03,Mike D Rogers,R,True,True,1193111,1218564
...,...,...,...,...,...,...,...
889,WI07,Tricia Zunker,D,False,False,1261957,1232690
890,WI08,Mike Gallagher,R,True,True,3202905,2841801
891,WI08,Amanda Stuck,D,False,False,416978,399916
892,WY01,Liz Cheney,R,True,True,3003883,3060167


In [4]:
# Read in source-of-funds data from Bonus
SOF = pd.read_csv('../data/us_rep_source_of_funds.csv')
SOF

,state,district,name,party,incumbent,winner,Type of Contribution,Amount,Percentage
0,Alabama,AL01,Jerry Carl,R,False,True,Small Individual Contributions (≤ $200),"$44,580",2.26%
1,Alabama,AL01,Jerry Carl,R,False,True,Large Individual Contributions,"$999,616",50.71%
2,Alabama,AL01,Jerry Carl,R,False,True,PAC Contributions*,"$387,000",19.63%
3,Alabama,AL01,Jerry Carl,R,False,True,Candidate self-financing,"$434,656",22.05%
4,Alabama,AL01,Jerry Carl,R,False,True,Other,"$105,470",5.35%
...,...,...,...,...,...,...,...,...,...
4465,Wyoming,WY01,Lynnette Grey Bull,D,False,False,Small Individual Contributions (≤ $200),"$64,222",47.71%
4466,Wyoming,WY01,Lynnette Grey Bull,D,False,False,Large Individual Contributions,"$65,975",49.02%
4467,Wyoming,WY01,Lynnette Grey Bull,D,False,False,PAC Contributions*,"$2,800",2.08%
4468,Wyoming,WY01,Lynnette Grey Bull,D,False,False,Candidate self-financing,$0,0.00%


## How often does the candidate who raised more money win a race? 

## How often does the candidate who spent more money win a race? 

## Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?

## How often does the incumbent candidate win a race?

## Can you detect any relationship between amount of money raised and the incumbent status of a candidate?